## 字符串输出解析器：StrOutputParser

In [3]:
# 1. 获取大模型
import os
import dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

# 2. 调用大模型
response = chat_model.invoke("什么是币安币？简要概述")
print(type(response))

# 3. 获取字符串的输出结果
# print(response.content)

parser = StrOutputParser()
output_str_response = parser.invoke(response)
print(output_str_response)

<class 'langchain_core.messages.ai.AIMessage'>
币安币（Binance Coin，简称BNB）是由全球知名的加密货币交易所币安（Binance）发行的一种数字资产。最初，BNB的用途主要是用于支付交易手续费，用户可以使用BNB支付手续费时享受折扣。随着币安生态系统的发展，BNB的应用场景逐渐扩展，包括但不限于：

1. **交易手续费**：在币安平台上使用BNB支付交易手续费可享受折扣。
2. **平台服务**：BNB可以用于参与币安的各种活动，例如首次发行代币（IEO）、流动性挖掘等。
3. **支付和消费**：BNB可以在多个商家和服务中作为支付手段使用。
4. **DeFi和Staking**：BNB可以用于在币安智能链（BSC）上参与去中心化金融（DeFi）项目，以及质押赚取收益。

随着币安不断扩展其产品和服务，BNB的应用也在不断增加。币安币采用了通缩机制，定期回购并销毁一定数量的BNB，这有助于控制供应量并提升其价值。
